# Quantum Information Foundations

This notebook demonstrates mastery of quantum information theory fundamentals essential for building a quantum autoencoder.

**Learning Objectives:**
1. Create and manipulate entangled quantum states (Bell states, GHZ states)
2. Compute density matrices and partial traces
3. Calculate entanglement entropy
4. Implement fidelity calculations
5. Understand why fidelity is the correct loss metric for quantum autoencoders

---

In [ ]:
# Import required libraries
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace, state_fidelity, entropy
from qiskit.visualization import plot_bloch_multivector, plot_state_city
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

print("Qiskit imported successfully!")
print(f"NumPy version: {np.__version__}")

## 1. Creating Entangled States

### 1.1 Bell States

The four Bell states are maximally entangled two-qubit states:

$$
\begin{align}
|\Phi^+\rangle &= \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) \\
|\Phi^-\rangle &= \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle) \\
|\Psi^+\rangle &= \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle) \\
|\Psi^-\rangle &= \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)
\end{align}
$$

In [ ]:
def create_bell_state(state_type='phi_plus'):
    """
    Create one of the four Bell states.
    
    Args:
        state_type: 'phi_plus', 'phi_minus', 'psi_plus', or 'psi_minus'
    
    Returns:
        Statevector: The requested Bell state
    """
    qc = QuantumCircuit(2)
    
    if state_type == 'phi_plus':
        # |Φ⁺⟩ = (|00⟩ + |11⟩)/√2
        qc.h(0)
        qc.cx(0, 1)
    elif state_type == 'phi_minus':
        # |Φ⁻⟩ = (|00⟩ - |11⟩)/√2
        qc.h(0)
        qc.z(0)
        qc.cx(0, 1)
    elif state_type == 'psi_plus':
        # |Ψ⁺⟩ = (|01⟩ + |10⟩)/√2
        qc.h(0)
        qc.cx(0, 1)
        qc.x(1)
    elif state_type == 'psi_minus':
        # |Ψ⁻⟩ = (|01⟩ - |10⟩)/√2
        qc.h(0)
        qc.z(0)
        qc.cx(0, 1)
        qc.x(1)
    else:
        raise ValueError(f"Unknown Bell state: {state_type}")
    
    return Statevector(qc)

# Create all four Bell states
bell_states = {
    'Φ⁺': create_bell_state('phi_plus'),
    'Φ⁻': create_bell_state('phi_minus'),
    'Ψ⁺': create_bell_state('psi_plus'),
    'Ψ⁻': create_bell_state('psi_minus')
}

# Display the Bell states
for name, state in bell_states.items():
    print(f"\n{name}: {state.data}")

In [ ]:
# Visualize |Φ⁺⟩ Bell state
phi_plus = bell_states['Φ⁺']
print("Visualizing |Φ⁺⟩ Bell state:")
display(plot_state_city(phi_plus, title='|Φ⁺⟩ State'))

### 1.2 GHZ States

The Greenberger-Horne-Zeilinger (GHZ) state is a maximally entangled state of three or more qubits:

$$
|GHZ_n\rangle = \frac{1}{\sqrt{2}}(|0\rangle^{\otimes n} + |1\rangle^{\otimes n})
$$

For 3 qubits: $|GHZ_3\rangle = \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle)$

In [ ]:
def create_ghz_state(n_qubits=3):
    """
    Create a GHZ state on n qubits.
    
    Args:
        n_qubits: Number of qubits (≥2)
    
    Returns:
        Statevector: The GHZ state
    """
    if n_qubits < 2:
        raise ValueError("GHZ state requires at least 2 qubits")
    
    qc = QuantumCircuit(n_qubits)
    
    # Create superposition on first qubit
    qc.h(0)
    
    # Entangle all qubits using CNOT gates
    for i in range(1, n_qubits):
        qc.cx(0, i)
    
    return Statevector(qc)

# Create GHZ states of different sizes
ghz_3 = create_ghz_state(3)
ghz_4 = create_ghz_state(4)

print("GHZ₃ state:")
print(ghz_3.data)
print("\nGHZ₄ state (first 8 amplitudes):")
print(ghz_4.data[:8])

### 1.3 Random Product States (Non-Entangled)

For comparison, we'll also create product states which are **not entangled**:

$$
|\psi_{product}\rangle = |\psi_1\rangle \otimes |\psi_2\rangle \otimes \cdots \otimes |\psi_n\rangle
$$

In [ ]:
def create_random_product_state(n_qubits=3):
    """
    Create a random product state (NOT entangled).
    
    Args:
        n_qubits: Number of qubits
    
    Returns:
        Statevector: A product state
    """
    qc = QuantumCircuit(n_qubits)
    
    # Apply random single-qubit rotations (no entangling gates)
    for i in range(n_qubits):
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2*np.pi)
        qc.u(theta, phi, 0, i)
    
    return Statevector(qc)

# Create a random product state
product_state = create_random_product_state(3)
print("Random product state:")
print(product_state.data)

## 2. Density Matrices and Partial Trace

### 2.1 Density Matrix Representation

A density matrix is a more general representation of quantum states:

$$
\rho = |\psi\rangle\langle\psi|
$$

For mixed states: $\rho = \sum_i p_i |\psi_i\rangle\langle\psi_i|$

**Key Properties:**
- Hermitian: $\rho = \rho^\dagger$
- Positive semidefinite: all eigenvalues ≥ 0
- Trace 1: $\text{Tr}(\rho) = 1$
- Pure state if $\rho^2 = \rho$ (equivalently, $\text{Tr}(\rho^2) = 1$)

In [ ]:
def compute_density_matrix(state):
    """
    Compute the density matrix from a statevector.
    
    Args:
        state: Statevector
    
    Returns:
        DensityMatrix: ρ = |ψ⟩⟨ψ|
    """
    return DensityMatrix(state)

# Compute density matrices
rho_bell = compute_density_matrix(bell_states['Φ⁺'])
rho_ghz = compute_density_matrix(ghz_3)
rho_product = compute_density_matrix(product_state)

print("Density matrix of |Φ⁺⟩ Bell state:")
print(rho_bell.data)
print(f"\nTrace: {np.trace(rho_bell.data):.6f}")
print(f"Tr(ρ²): {np.trace(rho_bell.data @ rho_bell.data):.6f}")

### 2.2 Partial Trace

The partial trace allows us to "trace out" subsystems to get the reduced density matrix:

$$
\rho_A = \text{Tr}_B(\rho_{AB})
$$

**Critical for Quantum Autoencoders:** We use partial trace to isolate the "trash" qubits!

In [ ]:
def compute_reduced_density_matrix(state, subsystem_qubits):
    """
    Compute the reduced density matrix by tracing out other qubits.
    
    Args:
        state: Statevector or DensityMatrix
        subsystem_qubits: List of qubit indices to keep
    
    Returns:
        DensityMatrix: Reduced density matrix
    """
    rho = DensityMatrix(state)
    return partial_trace(rho, subsystem_qubits)

# Example: For Bell state |Φ⁺⟩, trace out qubit 1
rho_reduced = compute_reduced_density_matrix(bell_states['Φ⁺'], [1])
print("Reduced density matrix of Bell state (traced out qubit 1):")
print(rho_reduced.data)
print(f"\nTr(ρ²): {np.trace(rho_reduced.data @ rho_reduced.data).real:.6f}")
print("\n⚠️ Note: Tr(ρ²) < 1 indicates this is a MIXED state (entanglement!)")

# Compare with product state
rho_product_reduced = compute_reduced_density_matrix(product_state, [1])
print("\n\nReduced density matrix of product state (traced out qubit 1):")
print(rho_product_reduced.data)
print(f"\nTr(ρ²): {np.trace(rho_product_reduced.data @ rho_product_reduced.data).real:.6f}")
print("✓ Product states remain pure after partial trace!")

## 3. Entanglement Entropy

Von Neumann entropy quantifies entanglement:

$$
S(\rho) = -\text{Tr}(\rho \log_2 \rho) = -\sum_i \lambda_i \log_2 \lambda_i
$$

where $\lambda_i$ are the eigenvalues of $\rho$.

**Key Insight:**
- $S(\rho) = 0$ for pure states (no entanglement with environment)
- $S(\rho) > 0$ for mixed states (entangled with traced-out subsystem)
- Maximum entropy = $\log_2(d)$ where $d$ is the dimension

In [ ]:
def compute_entanglement_entropy(state, subsystem_qubits):
    """
    Compute the Von Neumann entropy of a reduced density matrix.
    
    Args:
        state: Statevector or DensityMatrix
        subsystem_qubits: Qubits to trace out
    
    Returns:
        float: Von Neumann entropy
    """
    rho_reduced = compute_reduced_density_matrix(state, subsystem_qubits)
    return entropy(rho_reduced, base=2)

# Compute entanglement entropy for different states
print("=" * 60)
print("ENTANGLEMENT ENTROPY COMPARISON")
print("=" * 60)

# Bell state
S_bell = compute_entanglement_entropy(bell_states['Φ⁺'], [1])
print(f"\n|Φ⁺⟩ Bell state: S = {S_bell:.6f} bits")
print("  → Maximally entangled (S = 1 for 2-qubit system)")

# GHZ state
S_ghz = compute_entanglement_entropy(ghz_3, [1, 2])
print(f"\n|GHZ₃⟩ state: S = {S_ghz:.6f} bits")
print("  → Maximally entangled")

# Product state
S_product = compute_entanglement_entropy(product_state, [1, 2])
print(f"\nProduct state: S = {S_product:.6f} bits")
print("  → No entanglement (S ≈ 0)")

print("\n" + "=" * 60)
print("⚡ KEY TAKEAWAY: Entanglement entropy reveals quantum correlations")
print("   that classical systems cannot capture!")
print("=" * 60)

## 4. Fidelity Calculations

### 4.1 State Fidelity

Fidelity measures the "closeness" of two quantum states:

$$
F(\rho, \sigma) = \left(\text{Tr}\sqrt{\sqrt{\rho}\sigma\sqrt{\rho}}\right)^2
$$

For pure states $|\psi\rangle$ and $|\phi\rangle$:

$$
F(|\psi\rangle, |\phi\rangle) = |\langle\psi|\phi\rangle|^2
$$

**Properties:**
- $0 \leq F \leq 1$
- $F = 1$ for identical states
- $F = 0$ for orthogonal states

In [ ]:
def calculate_fidelity(state1, state2):
    """
    Calculate the fidelity between two quantum states.
    
    Args:
        state1, state2: Statevector or DensityMatrix objects
    
    Returns:
        float: Fidelity F ∈ [0, 1]
    """
    return state_fidelity(state1, state2)

# Test fidelity calculations
print("=" * 60)
print("FIDELITY CALCULATIONS")
print("=" * 60)

# Same state
F_same = calculate_fidelity(bell_states['Φ⁺'], bell_states['Φ⁺'])
print(f"\nF(|Φ⁺⟩, |Φ⁺⟩) = {F_same:.6f}")
print("  ✓ Identical states have F = 1")

# Different Bell states
F_bell_diff = calculate_fidelity(bell_states['Φ⁺'], bell_states['Φ⁻'])
print(f"\nF(|Φ⁺⟩, |Φ⁻⟩) = {F_bell_diff:.6f}")
print("  → Different Bell states are orthogonal")

# Bell vs product state
F_bell_product = calculate_fidelity(bell_states['Φ⁺'], product_state)
print(f"\nF(|Φ⁺⟩, |product⟩) = {F_bell_product:.6f}")
print("  → Entangled vs non-entangled states have low fidelity")

# Create a slightly perturbed state
qc_perturb = QuantumCircuit(2)
qc_perturb.h(0)
qc_perturb.cx(0, 1)
qc_perturb.rz(0.1, 0)  # Small rotation
perturbed = Statevector(qc_perturb)

F_perturbed = calculate_fidelity(bell_states['Φ⁺'], perturbed)
print(f"\nF(|Φ⁺⟩, |Φ⁺ + noise⟩) = {F_perturbed:.6f}")
print("  → Small perturbations give high fidelity")

print("\n" + "=" * 60)

### 4.2 Fidelity vs Trace Distance

**Trace Distance:**

$$
D(\rho, \sigma) = \frac{1}{2}\text{Tr}|\rho - \sigma|
$$

**Relationship:**

$$
1 - \sqrt{F} \leq D \leq \sqrt{1 - F}
$$

Both are valid distance metrics, but **fidelity is preferred for quantum autoencoders** because:
1. More intuitive (overlap interpretation)
2. Directly related to measurement probabilities
3. Easier to compute for pure states

In [ ]:
def trace_distance(state1, state2):
    """
    Calculate the trace distance between two quantum states.
    
    Args:
        state1, state2: Statevector or DensityMatrix
    
    Returns:
        float: Trace distance D ∈ [0, 1]
    """
    rho1 = DensityMatrix(state1)
    rho2 = DensityMatrix(state2)
    
    diff = rho1.data - rho2.data
    eigenvalues = np.linalg.eigvalsh(diff)
    return 0.5 * np.sum(np.abs(eigenvalues))

# Compare fidelity and trace distance
print("=" * 60)
print("FIDELITY vs TRACE DISTANCE")
print("=" * 60)

test_pairs = [
    ("Same state", bell_states['Φ⁺'], bell_states['Φ⁺']),
    ("Different Bell", bell_states['Φ⁺'], bell_states['Φ⁻']),
    ("Perturbed", bell_states['Φ⁺'], perturbed)
]

for name, s1, s2 in test_pairs:
    F = calculate_fidelity(s1, s2)
    D = trace_distance(s1, s2)
    print(f"\n{name}:")
    print(f"  Fidelity:       F = {F:.6f}")
    print(f"  Trace Distance: D = {D:.6f}")
    print(f"  Relationship: 1-√F = {1-np.sqrt(F):.6f}, √(1-F) = {np.sqrt(1-F):.6f}")
    print(f"  Verify: {1-np.sqrt(F):.6f} ≤ {D:.6f} ≤ {np.sqrt(1-F):.6f} ✓")

print("\n" + "=" * 60)

## 5. Why Fidelity is the Correct Loss Metric

### 5.1 The Problem with Classical Metrics (MSE, MAE)

**Mean Squared Error:**
$$
\text{MSE} = \frac{1}{N}\sum_i |\psi_{\text{original}}[i] - \psi_{\text{reconstructed}}[i]|^2
$$

**Why MSE fails for quantum states:**

1. **Global Phase Invariance**: $|\psi\rangle$ and $e^{i\theta}|\psi\rangle$ are physically identical, but MSE treats them as different

2. **Not Observable**: MSE compares amplitudes which cannot be directly measured

3. **No Physical Meaning**: MSE doesn't correspond to any quantum mechanical quantity

4. **Measurement Statistics**: What matters is $|\langle\psi|\phi\rangle|^2$ (fidelity), not amplitude differences

In [ ]:
def mse_loss(state1, state2):
    """
    Calculate MSE between statevectors (WRONG for quantum states!).
    """
    v1 = state1.data
    v2 = state2.data
    return np.mean(np.abs(v1 - v2)**2)

def fidelity_loss(state1, state2):
    """
    Calculate fidelity-based loss (CORRECT for quantum states).
    """
    F = state_fidelity(state1, state2)
    return 1 - F

# Demonstrate the problem
print("=" * 60)
print("WHY FIDELITY IS THE CORRECT LOSS FOR QUANTUM AUTOENCODERS")
print("=" * 60)

# Original Bell state
original = bell_states['Φ⁺']

# Same state with global phase
qc_phase = QuantumCircuit(2)
qc_phase.h(0)
qc_phase.cx(0, 1)
qc_phase.global_phase = np.pi / 4  # Add global phase
with_phase = Statevector(qc_phase)

print("\n1. GLOBAL PHASE INVARIANCE TEST")
print("-" * 40)
mse_phase = mse_loss(original, with_phase)
fid_phase = fidelity_loss(original, with_phase)
print(f"States differ only by global phase:")
print(f"  MSE Loss:      {mse_phase:.6f}  ❌ (non-zero)")
print(f"  Fidelity Loss: {fid_phase:.6f}  ✓ (zero)")
print("\n  Conclusion: MSE incorrectly penalizes physically identical states!")

# Small rotation vs large rotation
print("\n2. SENSITIVITY TO PHYSICAL DIFFERENCES")
print("-" * 40)

qc_small = QuantumCircuit(2)
qc_small.h(0)
qc_small.cx(0, 1)
qc_small.rz(0.1, 0)  # Small physical change
small_diff = Statevector(qc_small)

qc_large = QuantumCircuit(2)
qc_large.h(0)
qc_large.cx(0, 1)
qc_large.rz(np.pi, 0)  # Large physical change
large_diff = Statevector(qc_large)

mse_small = mse_loss(original, small_diff)
fid_small = fidelity_loss(original, small_diff)
mse_large = mse_loss(original, large_diff)
fid_large = fidelity_loss(original, large_diff)

print(f"Small rotation (Rz(0.1)):")
print(f"  MSE Loss:      {mse_small:.6f}")
print(f"  Fidelity Loss: {fid_small:.6f}")
print(f"\nLarge rotation (Rz(π)):")
print(f"  MSE Loss:      {mse_large:.6f}")
print(f"  Fidelity Loss: {fid_large:.6f}")
print(f"\n  Fidelity correctly shows: {fid_large:.4f} > {fid_small:.4f}")

print("\n" + "=" * 60)
print("⚡ FINAL VERDICT:")
print("=" * 60)
print("\nFor Quantum Autoencoders, we MUST use:")
print("\n  L = 1 - F(ρ_original, ρ_reconstructed)")
print("\nBecause:")
print("  ✓ Respects quantum mechanics (phase invariance)")
print("  ✓ Corresponds to measurement statistics")
print("  ✓ Has physical interpretation")
print("  ✓ Directly optimizes quantum state overlap")
print("\n" + "=" * 60)

## 6. Summary: Quantum Information Foundations Checklist

✅ **Created Entangled States**
- Bell states: |Φ⁺⟩, |Φ⁻⟩, |Ψ⁺⟩, |Ψ⁻⟩
- GHZ states for n qubits
- Product states (non-entangled) for comparison

✅ **Computed Density Matrices**
- Converted statevectors to density matrices
- Verified properties: Tr(ρ) = 1, Hermitian, positive semidefinite
- Distinguished pure vs mixed states using Tr(ρ²)

✅ **Calculated Partial Traces**
- Traced out subsystems to get reduced density matrices
- Showed entangled states → mixed reduced states
- Showed product states → pure reduced states

✅ **Measured Entanglement Entropy**
- Computed Von Neumann entropy S(ρ)
- S > 0 for entangled states
- S ≈ 0 for product states

✅ **Implemented Fidelity Calculations**
- State fidelity F(ρ, σ)
- Compared with trace distance D(ρ, σ)
- Verified mathematical relationships

✅ **Proved Why Fidelity is the Correct Loss**
- Global phase invariance
- Physical observability
- Measurement statistics interpretation
- Superiority over MSE/MAE

---

## Next Steps

With these foundations, we can now:
1. Build a quantum autoencoder architecture
2. Design the loss function: $L = 1 - F(\rho_{\text{in}}, \rho_{\text{reconstructed}})$
3. Train using classical optimizers (COBYLA, SPSA)
4. Prove quantum advantage over classical ML

**If you can't explain why fidelity is the right loss, you're not ready to claim quantum advantage.** ✅